In [155]:
import os
from pydub import AudioSegment
from moviepy.editor import *
import wave
import math
import pyaudio
import numpy
import pandas as pd

In [156]:
video_path='v1.mp4'
voice_path='v1.mp3'
video = VideoFileClip(os.path.join(video_path))
video.audio.write_audiofile(os.path.join(voice_path))
sound = AudioSegment.from_mp3(voice_path)
sound.export("test1.wav", format="wav")

MoviePy - Writing audio in v1.mp3


MoviePy - Done.


<_io.BufferedRandom name='test1.wav'>

In [157]:
class VoiceDetect():
    def __init__(self,wav_path,save_path):
        self.wav_path = wav_path
        self.save_path = save_path

    #求数组平均值
    def mean(self,a):
        return numpy.longlong(sum(a)) / len(a)
    
    #把mp3格式转化为.wav模式
    def trans_mp3_2_wav(self,filename):
        name = os.path.splitext(filename)[0]
        audio=AudioSegment.from_mp3(filename)
        audio.export(name + ".wav",format="wav")
        
    def abslist(self,a):
        return list(map(abs,a))

    def recvoice(self,filename):
        #取文件名
        #打开WAV文档，文件路径根据需要做修改
        wf = wave.open(os.path.join(self.wav_path,filename+".wav"), "rb")
        #创建PyAudio对象
        p = pyaudio.PyAudio()
        stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),channels=wf.getnchannels(),rate=wf.getframerate(),output=True)
        nframes = wf.getnframes()
        framerate = wf.getframerate()
        #读取完整的帧数据到str_data中，这是一个string类型的数据
        str_data = wf.readframes(nframes)
        wf.close()
        #将波形数据转换为数组
        wave_data = numpy.fromstring(str_data, dtype=numpy.short)
        #wave_data=list(map(abs,wave_data))
        M = []
        #求60秒取样的平均值，若为双声道，再多乘2 
        gap = 1
        n = framerate*2*gap
        for i in range(0, len(wave_data), n):
            M.append(wave_data[i:i+n]/10) #传化成分贝
        M=map(self.abslist,M)
        sound = list(map(self.mean,M))
        #时间数组，与sound配对形成系列点坐标
        time = numpy.arange(0,nframes/(framerate*gap))
        time = time.astype(int) * gap
        #生成csv文件
        dataframe = pd.DataFrame({'Time':time,'Sound':sound})
        dataframe.to_csv(os.path.join(self.wav_path,filename+".csv"),index=False,sep=',')
        return dataframe
    
    def analyze(self, filename, df):
        percnet = 0.75
        threshold = df['Sound'].quantile(percnet)
        condi=df['Sound']>threshold

        # df[condi]['Time'].to_csv(os.path.join(self.save_path,filename+'.csv'))

        with open(os.path.join(self.save_path,filename+'.txt'),'w') as f:
            for row in df[condi]['Time']:
                f.writelines(str(row)+'\n')

    # 获取文件目录下的mp3 wav格式音频
    def file_name(self):
        L=[]
        print('start')
        for root, dirs,files in os.walk(self.wav_path):
            # print(f'file:{files}')
            # print(f'root:{root}')
            # print(f'dirs{dirs}')
            for file in files:
                #.mp3格式转化为.wav
                # print(f'files:{file}')
                # if os.path.splitext(file)[1] == '.mp3': 
                #     trans_mp3_2_wav(os.path.join(file_dir, file))
                #     print(file)
                #     L.append(os.path.join(file_dir, file[:-3]+'wav'))
                if os.path.splitext(file)[1] == '.wav':
                    L.append(file[:-4])
        for i in range(len(L)):
            print(L[i])
            df = self.recvoice(L[i])
            self.analyze(L[i],df)
        print('finish')

In [158]:
wav_path='G:\\Github\\spark_photo\\voice'
save_path = 'G:\\Github\\spark_photo\\voice\\dataset'

vd = VoiceDetect(wav_path,save_path)
vd.file_name()

start
test1
v1
finish
